In [16]:
from neo4j import GraphDatabase

url = 'neo4j://localhost:7687'
user = 'neo4j'
password = 'securepassword'

driver = GraphDatabase.driver(url, auth=(user, password))

1. Найти самый старый фильм и самый новый фильм - вывести их названия по 1 шт (2 запроса)

In [1]:
def min_films(tx):
    min_film = tx.run(
        """
    MATCH (m:Movie)
    Return MIN(m.released) as min_movie 
"""
    )
    record = min_film.single()
    return record["min_movie"] if record else None

In [5]:
def max_films(tx):
    max_film = tx.run(
        """
    MATCH (m:Movie)
    Return MAX(m.released) as max_movie 
"""
    )
    record = max_film.single()
    return record["max_movie"] if record else None

In [4]:
with driver.session() as session:
    result = session.execute_read(min_films)
    print(result)

1975


In [6]:
with driver.session() as session:
    result = session.execute_read(max_films)
    print(result)

2012


2. Получить среднее количество актёров на фильм 

In [10]:
def avg_person_onfilm(tx, film):
    avg_person = tx.run(
        """
    MATCH (m:Movie{title:$film})<-[:ACTED_IN]-(p:Person)
WITH m, COUNT(p) AS actors_count
RETURN AVG(actors_count) AS avg_act_movie
""",
film=film
    )
    record = avg_person.single()
    return record["avg_act_movie"] if record else None

In [13]:
with driver.session() as session:
    record = session.execute_read(avg_person_onfilm, "What Dreams May Come")
    print(record)

5.0


3. Группировка фильмов по годам и подсчёт количества фильмов в каждом году

In [14]:
def group_film_by_year(tx):
    group_film = tx.run(
        """
        MATCH (movie:Movie)
        RETURN movie.released, count(movie) as count_movie
        ORDER BY movie.released        
"""
    )
    return list(group_film)

In [22]:
with driver.session() as session:
    record = session.execute_read(group_film_by_year)
    print(record)

    for p in record:
        print(f'{p["movie.released"]} - {p["count_movie"]}')

[<Record movie.released=1975 count_movie=3>, <Record movie.released=1986 count_movie=6>, <Record movie.released=1990 count_movie=3>, <Record movie.released=1992 count_movie=12>, <Record movie.released=1993 count_movie=3>, <Record movie.released=1995 count_movie=6>, <Record movie.released=1996 count_movie=9>, <Record movie.released=1997 count_movie=6>, <Record movie.released=1998 count_movie=9>, <Record movie.released=1999 count_movie=12>, <Record movie.released=2000 count_movie=9>, <Record movie.released=2003 count_movie=9>, <Record movie.released=2004 count_movie=3>, <Record movie.released=2006 count_movie=9>, <Record movie.released=2007 count_movie=3>, <Record movie.released=2008 count_movie=6>, <Record movie.released=2009 count_movie=3>, <Record movie.released=2012 count_movie=3>]
1975 - 3
1986 - 6
1990 - 3
1992 - 12
1993 - 3
1995 - 6
1996 - 9
1997 - 6
1998 - 9
1999 - 12
2000 - 9
2003 - 9
2004 - 3
2006 - 9
2007 - 3
2008 - 6
2009 - 3
2012 - 3
